1. Use `emails.csv` instead of `emails_binary.csv` in your experiments. Think about adjustments to be made to the joint and conditional probability formulas. `emails.csv` is upload on Moodle under PA.4. 


2. Include $\alpha$ in equation 2 for classification: 

$$ \alpha~𝑃~(\text{spam}| 𝑤𝑜𝑟𝑑1, 𝑤𝑜𝑟𝑑2,...𝑤𝑜𝑟𝑑𝑛)> \alpha~𝑃~( ¬ \text{spam}| 𝑤𝑜𝑟𝑑1, 𝑤𝑜𝑟𝑑2,... 𝑤𝑜𝑟𝑑𝑛) $$

3. Identify top 10 words most indicative of an email being Spam and top 10 words most indicative of an email being Ham. 

4. Use common-sense preprocessing steps to improve your accuracy scores. 

In [2]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.4./emails.csv', index_col=0)
data.head()

,the,to,ect,and,for,of,a,you,hou,in,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
Email No.,,,,,,,,,,,,,,,,,,,,,
Email 1,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Email 2,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,1,0,0
Email 3,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
Email 4,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
Email 5,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,1,0,0


In [118]:
# badwords = data[data['Prediction'] != data['myPreds']]
# badWordCount = list(badwords)


totalWordSum = data[data.columns[:-1]].sum()
# totalWordSum = 6,026,877

pSpam = data['Prediction'].mean()
pHam = 1 - pSpam

word_counts = data.groupby('Prediction').sum()

cp1 = word_counts.loc[1] / totalWordSum
cp3 = word_counts.loc[0] / totalWordSum

# cp1 = removedWords.loc[1] / totalWordSum
# cp3 = removedWords.loc[0] / totalWordSum


def get_prediction(email):
  words_in_email = email[email == 1].index 
  scoreSpam = cp1.loc[words_in_email].prod() * pSpam
  scoreHam = cp3.loc[words_in_email].prod() * pHam
  alpha = 1 / (scoreSpam + scoreHam)

  my_prediction = 1 if (scoreSpam * alpha) > (scoreHam * alpha) else 0 #ternary
  return my_prediction

data['myPreds'] = data[data.columns[:-2]].apply(get_prediction, axis = 1)

accuracy = (data['Prediction'] == data['myPreds']).sum() / len(data)
#accuracy, totalWordSum.sum(), word_counts
accuracy


0.832753286929621

# attempted increase of prediction accuracy:

#  1. find n words with the worst standard deviation in the incorrectly predicted emails
#  2. then remove them and retry
#  3. I am not good enough with Pandas to be able to do all the stuff I would like to try, but I got close here


In [116]:
badEmails = data[data['Prediction'] != data['myPreds']]
badWordCount = list(badEmails)

#badwords.describe()
sortedBad = badEmails.iloc[:,(-badEmails.std()).argsort()] # sort the df of bad words by std
threshold = 2.0

underTwoStd = sortedBad.drop(sortedBad.std()[sortedBad.std() < threshold].index.values, axis=1) # get rid of cols with high std
badList = list(underTwoStd) # list of words under 2 std

removedHighSTDCols = badEmails.drop(badList, axis = 1)  # refined dataframe of emails we predicted wrong originally
removedRowsNotPredicted = data.drop(data[data['Prediction'] != data['myPreds']].index) # original dataframe without the incorrect email predictions

    # combine the df of incorrectly predicted emails with emails predicted correctly
combined_highStd_and_original_df = pd.concat((removedHighSTDCols,removedRowsNotPredicted)).groupby('Email No.').first().reset_index()
combined_highStd_and_original_df = combined_highStd_and_original_df.groupby('Prediction').sum()


totalWordSum = data[data.columns[:-1]].sum() # totalWordSum = 6,026,877

pSpam = data['Prediction'].mean()
pHam = 1 - pSpam

word_counts = data.groupby('Prediction').sum()

cp1 = (combined_highStd_and_original_df.loc[1] / totalWordSum) +1 # ends up as zero sometimes due to removing words earlier, adding 1 to everything to avoid 
                                                                  # (assuming in long run it all evens out since it adds less than a percent of accuracy,)
cp3 = (combined_highStd_and_original_df.loc[0] / totalWordSum) +1 #  regardless this does not improve the accuracy at all)


def get_prediction(email):
  words_in_email = email[email == 1].index 
  scoreSpam = cp1.loc[words_in_email].prod() * pSpam
  scoreHam = cp3.loc[words_in_email].prod() * pHam
  alpha = 1 / (scoreSpam + scoreHam)

  my_prediction = 1 if (scoreSpam * alpha) > (scoreHam * alpha) else 0 #ternary
  return my_prediction

data['myPreds'] = data[data.columns[:-2]].apply(get_prediction, axis = 1)

accuracy = (data['Prediction'] == data['myPreds']).sum() / len(data)
accuracy

0.7182907965970611

# 10 worst words (highest std in the incorrectly predicted emails)
# 10 best words (lowest std in the correctly predicted emails)


In [117]:
badwords = data[data['Prediction'] != data['myPreds']]
badWordCount = list(badwords)

sortedBad = badwords.iloc[:,(-badwords.std()).argsort()] # sort the df of bad words by std, can now just grab the first 10
badList = []
i = 0
while i < 10:
  badList.append( list(sortedBad)[i] )
  i+=1

###############################################################################
goodwords = data[data['Prediction'] == data['myPreds']]
goodWordCount = list(goodwords)

sortedGood = goodwords.iloc[:,(-goodwords.std()).argsort()]
goodList = []
i = 1
while i < 11:
  goodList.append( list(sortedGood)[-i] )
  i+=1

print("The top Ten most predictive words are", goodList)
print("The top Ten least predictive words are", badList)

The top Ten most predictive words are ['studio', 'moopid', 'spacer', 'netherlands', 'palestinian', 'abdv', 'impotence', 'php', 'knle', 'pharmacy']
The top Ten least predictive words are ['e', 't', 'a', 'i', 'o', 'n', 'r', 's', 'l', 'c']


In [94]:
#scratchwork

# badwords = data[data['Prediction'] != data['myPreds']]
# #badwords.describe()
# sortedBad = badwords.iloc[:,(-badwords.std()).argsort()]
# threshold = 2.0

# underTwoStd = sortedBad.drop(sortedBad.std()[sortedBad.std() < threshold].index.values, axis=1)



badwords = data[data['Prediction'] != data['myPreds']]
badWordCount = list(badwords)

#badwords.describe()
sortedBad = badwords.iloc[:,(-badwords.std()).argsort()]
threshold = 2.0

overTwoStd = sortedBad.drop(sortedBad.std()[sortedBad.std() < threshold].index.values, axis=1)
badList = list(overTwoStd)
refinedList = badwords.drop(badList, axis = 1)
refinedList #refined list of emails we predicted wrong originally

removedRows = data.drop(data[data['Prediction'] != data['myPreds']].index)
# finished = removedRows.merge(refinedList, on='Email No.')
finished = pd.concat((refinedList,removedRows)).groupby('Email No.').first().reset_index()
finished = finished.groupby('Prediction').sum()
finished


,hou,enron,will,have,by,from,gas,deal,meter,hpl,...,ut,mi,po,sp,ea,le,der,lls,ur,ff
Prediction,,,,,,,,,,,,,,,,,,,,,
0,9532,6906,3663,3257,2475,3215,2945,3560,2736,3281,...,4717.0,5598.0,4915.0,2695.0,13486.0,14979.0,1712.0,326.0,8572.0,2469.0
1,938,0,738,905,925,995,248,239,47,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<center>
<img width="50%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/programming_assignments/PA.4./Always-Has-Been.png">
</center>